<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/Pandas_Regression_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from google.colab import files
uploaded = files.upload()

Saving weatherHistory.csv to weatherHistory.csv


In [0]:
weather_hist = pd.read_csv('weatherHistory.csv')

In [0]:
weather_hist['Wind Speed (km/h)'].max()

63.8526

Найдём день, когдабыл самый сильный ветер:

In [0]:
weather_hist[weather_hist['Wind Speed (km/h)'] == 63.8526]['Formatted Date']

12181    2007-01-29 13:00:00.000 +0100
Name: Formatted Date, dtype: object

In [0]:
weather_hist['Temperature (C)'].min()

-21.822222222222226

А теперь выясним, какой же день был самым холодным:

In [0]:
weather_hist[weather_hist['Temperature (C)'] == -21.822222222222226]['Formatted Date']

54847    2012-02-10 07:00:00.000 +0100
Name: Formatted Date, dtype: object

In [0]:
weather_hist['Formatted Date']=pd.to_datetime(weather_hist['Formatted Date'], utc=True)

In [0]:
weather_hist['year']=weather_hist['Formatted Date'].dt.year

In [0]:
weather_hist['month']=weather_hist['Formatted Date'].dt.month

In [0]:
weather_hist['week']=weather_hist['Formatted Date'].dt.week

In [0]:
weather_hist['day']=weather_hist['Formatted Date'].dt.day

In [0]:
weather_hist['mon']=weather_hist['Formatted Date'].dt.to_period('M')

/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [0]:
weather_hist['wee']=weather_hist['Formatted Date'].dt.to_period('W')

/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [0]:
set(weather_hist[weather_hist['Summary'] == 'Rain']['year'])

{2016}

Исходя из этих данных — дождь за всё время наблюдения был только в 2016 году. Проверим для кратковременных дождей:

In [0]:
set(weather_hist[weather_hist['Summary'] == 'Light Rain']['year'])

{2012, 2016}

Значит, дождя в 2014 году не было

Сгруппируем самую большую скорость ветра по месяцам и отсортируем от самого ветренного — к наименее ветренному:

In [0]:
columns_to_show = ['Wind Speed (km/h)']
weather_hist.groupby('mon')[columns_to_show].sum().sort_values(by='Wind Speed (km/h)', ascending=False)

,Wind Speed (km/h)
mon,
2009-03,12560.4794
2008-03,10879.1242
2010-03,10870.0277
2013-03,10766.3759
2006-03,10729.5874
...,...
2015-08,5519.7562
2006-07,5352.2679
2011-11,4947.5783


Теперь найдём самый влажный год:

In [0]:
columns_to_show = ['Humidity']
weather_hist.groupby('year')[columns_to_show].sum().sort_values(by='Humidity', ascending=False)

,Humidity
year,
2010,6999.63
2006,6721.98
2016,6682.70
2013,6606.77
2014,6555.24
2011,6447.43
2015,6415.42
2009,6195.48
2008,6159.68


Теперь найдём неделю с самой переменчивой погодой:

In [0]:
weather_hist['alltogether'] = weather_hist[['Temperature (C)','Wind Speed (km/h)','Humidity']].sum(axis=1)
grouped_week = weather_hist.groupby(['wee']).aggregate(np.std)
grouped_week.head()

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),year,month,week,day,alltogether
wee,,,,,,,,,,,,,
2005-12-26/2006-01-01,2.112555,2.445988,0.041932,3.951112,5.468699,1.941970,0.0,2.452614,0.2,2.2,0.0,6.000000,5.343694
2006-01-02/2006-01-08,2.218176,2.340866,0.064341,5.832611,118.865848,3.474981,0.0,136.196711,0.0,0.0,0.0,2.005979,7.122613
2006-01-09/2006-01-15,1.801108,2.358477,0.088806,2.458327,119.794358,2.969237,0.0,112.624343,0.0,0.0,0.0,2.005979,3.136823
2006-01-16/2006-01-22,3.118158,4.374815,0.077038,8.004107,93.184172,3.149746,0.0,7.785396,0.0,0.0,0.0,2.005979,8.288142
2006-01-23/2006-01-29,5.014988,5.976073,0.120662,5.140100,87.530862,2.718369,0.0,5.328491,0.0,0.0,0.0,2.005979,7.102721


Неделя с самой переменчивой погодой это:

In [0]:
grouped_week['alltogether'].idxmax()

Period('2007-01-29/2007-02-04', 'W-SUN')

Второе задание:

In [0]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [0]:
data = pd.read_csv('data.csv')

In [0]:
data.shape

(138715, 18)

In [0]:
data.dropna(subset=['description'], inplace=True)
data = pd.read_csv('data.csv')

In [0]:
import re
from sklearn.metrics import mean_squared_error

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
y_train, y_test = train_test_split(data.deal_probability, random_state=42)

In [0]:
np.random.rand(len(y_test))

array([0.6756927 , 0.47526568, 0.03003784, ..., 0.4279017 , 0.48635587,
       0.34150346])

In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))

Возьмём также в качестве признаков: 1) прилагательные, которые будут положительно описывать товар (что будет повышать качества товара в глазах клиента); 2) слова на латинице (скорее всего, это будут названия известных брендов, что тоже должно положительно сказаться на вероятности покупки); 3) слова, построенные по модели "буквы+число" или "буквы+число+число" (это моугт быть указания на модель товара, что позволит покупателю лучше понимать, что за товар продаётся); 4) просто числа (год изготовления, модель, количество и т.д. — эти признаки также должны повышать интерес к объявлению); 5) Символы "!", ":)", ")))", которые выражают какие-то эмоции и привлекают внимание потенциальных покупателей.


In [0]:
adjectives = ['новый', "новая", "хороший", "хорошем", "хорошая", "отличный", "отличном", "отличная", "рабочий", "рабочем", "рабочая", "идеальный", "идеальном", "идеальная", "супер"]
data['number_of_adjectives'] = data['description'].apply(lambda x: len([ch for ch in x if ch in adjectives]))
data['number_of_latins'] = data['description'].apply(lambda x: len(re.findall("[a-zA-Z]+", x)))
data['number_of_series'] = data['description'].apply(lambda x: len(re.findall("([а-яА-Яa-zA-Z]+)([0-9]+)([а-яА-Яa-zA-Z]+)?", x)))
data['number_of_numbers'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isnumeric()]))
data['number_of_emotions'] = data['description'].apply(lambda x: len([ch for ch in x if '!'in x or ':)' in x or ')))' in x]))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
X = cv.fit_transform(data.description)
y = data.deal_probability.values

In [0]:
from sklearn.linear_model import Ridge

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = Ridge()
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2513093030154705

In [0]:
regressor = Ridge(alpha=10)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2512515450632489

In [0]:
regressor = Ridge(alpha=100)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2510952817990643

In [0]:
regressor = Ridge(alpha=1000)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2519807350013394

In [0]:
regressor = Ridge(alpha=0.5)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2513142052006059

In [0]:
regressor = Ridge(alpha=0.000001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2513195832532735

In [0]:
cv = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.4)
X = cv.fit_transform(data.description)
y = data.deal_probability.values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.24837197234438146

In [0]:
regressor = Ridge(alpha=10)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.248098946453737

In [0]:
regressor = Ridge(alpha=100)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2492990583147276

In [0]:
regressor = Ridge(alpha=50)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2484038456052774

In [0]:
regressor = Ridge(alpha=15)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2480589222467377

Выше попробовал подобрать разные параметры alpha, но улучшить качество (0.248) не получилось

In [0]:
from sklearn.linear_model import SGDRegressor

In [0]:
regressor = SGDRegressor(max_iter=10, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2524147254796756

In [0]:
from scipy.sparse import hstack

In [0]:
cv = CountVectorizer(max_features=100, min_df=5, max_df=0.4, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=100, min_df=5,analyzer='char', ngram_range=(3,3), max_df=0.4)

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])

In [0]:
X_cv.shape, X_tfidf.shape

((138715, 100), (138715, 100))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [59]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2573235283442644

In [60]:
regressor = SGDRegressor(max_iter=10, alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26286192616371035

In [61]:
regressor = SGDRegressor(max_iter=50, alpha=100)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26371289202995735

In [62]:
regressor = SGDRegressor(max_iter=100, alpha=100)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26371477498241264

Выше я попробовал также менять параметр alpha, а также изменил max_features до 100 и попробовал разные max_iter. Результат лучше не стал.

In [0]:
from sklearn.metrics import r2_score

In [64]:
r2_score(y_test, preds)

-0.00030358488661574867